In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from evaluate import load

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load and preprocess E2E NLG dataset
def load_and_preprocess_e2e():
    data = load_dataset("e2e_nlg")

    def preprocess_e2e(example):
        input_text = "MR: " + example['meaning_representation']
        target_text = "Output: " + example['human_reference']
        return {
            "input_text": input_text,
            "target_text": target_text
        }

    processed_data = data.map(preprocess_e2e)

    def tokenize_function(example):
        return tokenizer(
            example['input_text'],
            text_target=example['target_text'],
            truncation=True,
            padding="max_length",
            max_length=128
        )

    # Tokenize datasets
    tokenized_train = processed_data['train'].map(tokenize_function, batched=True)
    tokenized_val = processed_data['validation'].map(tokenize_function, batched=True)

    return tokenized_train, tokenized_val

# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model = model.to(device)

# Load datasets
tokenized_train, tokenized_val = load_and_preprocess_e2e()

# LoRA configuration
peft_config = LoraConfig(
    task_type=TaskType.SEQ2SEQ_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

# Apply LoRA to the model
model = get_peft_model(model, peft_config)
model = model.to(device)

# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model
)

# Load evaluation metrics
bleu = load("bleu")
nist = load("nist_mt")
meteor = load("meteor")
rouge = load("rouge")
cider = load("cider")

# Compute metrics
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(logits, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    bleu_score = bleu.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    nist_score = nist.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    meteor_score = meteor.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    rouge_score = rouge.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    cider_score = cider.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])

    return {
        "bleu": bleu_score["bleu"],
        "nist": nist_score["nist_mt"],
        "meteor": meteor_score["meteor"],
        "rouge_l": rouge_score["rougeL"].mid.fmeasure,
        "cider": cider_score["CIDEr"]
    }

# Training arguments
training_args = TrainingArguments(
    output_dir="./results_e2e",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs_e2e",
    logging_steps=10,
    save_strategy="epoch"
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Validation Results: {eval_results}")

# Save the model
model.save_pretrained("lora_gpt2_e2e")
print("Model saved.")